<a href="https://colab.research.google.com/github/abhijeetraj22/Cogitativum_Short_Assignment/blob/main/SA_Tasks_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cogitativum Ventures Private Limited
## Short Assignment
## 1.Let us know how you can write data from an API into Pandas Dataframe

### Step 1: Import Libraries

In [1]:
import pandas as pd
import requests
import json

### Step 2: Make a GET Request from an API Endpoint

In [2]:
url = "https://api.covid19api.com/summary"
json_Content = requests.get(url)


### Step 3: Extract Data from the Request using the Json() Method

In [3]:
json_data = json_Content.json()
#for Arrange json data
#json_content = json.dumps(json_data, indent = 4, sort_keys=True)
#print(json_content)
json_data

{'Countries': [{'Country': 'Afghanistan',
   'CountryCode': 'AF',
   'Date': '2021-08-04T17:53:29.243Z',
   'ID': 'bdb1265d-a926-43c4-8680-ac035de3603b',
   'NewConfirmed': 0,
   'NewDeaths': 0,
   'NewRecovered': 0,
   'Premium': {},
   'Slug': 'afghanistan',
   'TotalConfirmed': 148572,
   'TotalDeaths': 6804,
   'TotalRecovered': 82586},
  {'Country': 'Albania',
   'CountryCode': 'AL',
   'Date': '2021-08-04T17:53:29.243Z',
   'ID': 'eff04263-ec47-4367-8310-b58905602383',
   'NewConfirmed': 0,
   'NewDeaths': 0,
   'NewRecovered': 0,
   'Premium': {},
   'Slug': 'albania',
   'TotalConfirmed': 133211,
   'TotalDeaths': 2457,
   'TotalRecovered': 130291},
  {'Country': 'Algeria',
   'CountryCode': 'DZ',
   'Date': '2021-08-04T17:53:29.243Z',
   'ID': '5066ec8b-2052-4087-97ed-4ef35b559c7f',
   'NewConfirmed': 0,
   'NewDeaths': 0,
   'NewRecovered': 0,
   'Premium': {},
   'Slug': 'algeria',
   'TotalConfirmed': 175229,
   'TotalDeaths': 4370,
   'TotalRecovered': 117557},
  {'Country

### Step 4: Exploring our JSON using the Keys() and Type() Methods

In [4]:
json_data.keys()

dict_keys(['ID', 'Message', 'Global', 'Countries', 'Date'])

In [11]:
json_data['Global']

{'Date': '2021-08-04T17:53:29.243Z',
 'NewConfirmed': 306965,
 'NewDeaths': 4359,
 'NewRecovered': 99209,
 'TotalConfirmed': 199299604,
 'TotalDeaths': 4243569,
 'TotalRecovered': 130552703}

In [6]:
json_data['Date']

'2021-08-04T17:53:29.243Z'

In [7]:
print("Global date type ",type(json_data['Global']))
print("Countries data type ",type(json_data['Countries']))
print("Date data type ",type(json_data['Date']))

Global date type  <class 'dict'>
Countries data type  <class 'list'>
Date data type  <class 'str'>


### Step 5: JSON to Dataframe

In [8]:
df = pd.DataFrame(json_data['Countries'])

In [9]:
type(df)

pandas.core.frame.DataFrame

In [10]:
df[df["Country"]=='India']

,ID,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
76,f8ee3b2f-2e14-43de-bf2f-5421395f97d9,India,IN,india,42625,31769132,562,425757,36668,30933022,2021-08-04T17:53:29.243Z,{}


## 2. How you can batch insert into a PostgreSQL database a Pandas dataframe.

### Step 1: Prerequisites
- Python 3.8.3 : [Anaconda download link](https://www.anaconda.com/products/individual)
- PostgreSQL 13 : [Download link](https://www.postgresql.org/download/)
- Psycopg2 : To install **Psycopg2** use the command: **pip install psycopg2**

In [ ]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


### Step 2: Import Libraries

In [ ]:
import sys
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras
import pandas as pd


### Step 3: Import Dataset

In [ ]:
iris_Data = pd.read_csv('https://raw.githubusercontent.com/abhijeetraj22/TSF-GRIP_Tasks/main/Datasets/Iris.csv',index_col=False)
iris_Data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
iris_Data.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [ ]:
type(iris_Data)

pandas.core.frame.DataFrame

### Step 4 : Specify the connection parameters

In [ ]:
conn_params_dic = {
    "host"      : "localhost",
    "database"  : "postgres",
    "user"      : "postgres",
    "password"  : "postgres"
}

### Step 5 : Support Functions

In [ ]:
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    line_n = traceback.tb_lineno    
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    print ("\nextensions.Diagnostics:", err.diag)    
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
    
# Define a connect function for PostgreSQL database server
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successful..................")
        
    except OperationalError as err:
        show_psycopg2_exception(err)        
        conn = None
    return conn

# Define function using psycopg2.extras.execute_batch() to insert the dataframe
def execute_batch(conn, datafrm, table, page_size=150):
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_batch(cursor, sql, tpls, page_size)
        conn.commit()
        print("Data inserted using execute_batch() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        show_psycopg2_exception(err)
        cursor.close()

###  Step 6: Connect to the database & batch insert into a PostgreSQL database

In [ ]:

conn = connect(conn_params_dic)
conn.autocommit = True

if conn!=None:
    
    try:
        cursor = conn.cursor();
        
        # Dropping table iris if exists
        cursor.execute("DROP TABLE IF EXISTS iris;")
        
        sql = '''CREATE TABLE iris(
        sepal_length DECIMAL(2,1) NOT NULL, 
        sepal_width DECIMAL(2,1) NOT NULL, 
        petal_length DECIMAL(2,1) NOT NULL, 
        petal_width DECIMAL(2,1),
        species CHAR(11)NOT NULL
        )'''
        
        cursor.execute(sql);
        print("iris table is created successfully................")
        
        # Run the execute_batch method
        execute_batch(conn, iris_Data,'iris')
    
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        show_psycopg2_exception(err)
        conn = None


Connecting to the PostgreSQL...........
Connection successful..................
iris table is created successfully................
Data inserted using execute_batch() successfully...


### Step 7: Query the database to check our work
let’s query the database to make sure that our inserted data has been saved correctly.

In [ ]:
conn = connect(conn_params_dic)
sql = "SELECT * FROM iris" 
cursor = conn.cursor()
cursor.execute(sql)
tuples = cursor.fetchall()
cols = list(iris_Data.columns)
irisdf = pd.DataFrame(tuples,columns=cols) 
cursor.close()
conn.close()
irisdf.tail()

Connecting to the PostgreSQL...........
Connection successful..................


,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [ ]:
type(irisdf)

pandas.core.frame.DataFrame